In [10]:
from modules.preprocess_cols import preprocess_for_tableau
import pandas as pd

df_raw = pd.read_csv("telco.csv")
df_tableau, le = preprocess_for_tableau(df_raw)
df_tableau.to_csv("telco_tableau.csv", index=False)

print("✅ telco_tableau.csv 저장 완료!")

✅ telco_tableau.csv 저장 완료!


In [2]:
import pandas as pd

# 단계별 고객 수
step_0 = 7043
step_1 = 1051
step_2 = 1051
step_3 = 967
step_4 = 403
step_5 = 403

# Sankey용 데이터 프레임 구성
sankey_df = pd.DataFrame({
    'Source': [
        '1. 전체 고객',
        '2. 가입 3개월 이하',
        '3. 인터넷 있음',
        '4. 보안 없음',
        '5. 만족도 낮음 (≤2점)'
    ],
    'Target': [
        '2. 가입 3개월 이하',
        '3. 인터넷 있음',
        '4. 보안 없음',
        '5. 만족도 낮음 (≤2점)',
        '6. 최종 이탈'
    ],
    'Value': [
        step_1,
        step_2,
        step_3,
        step_4,
        step_5
    ]
})


# CSV 저장
sankey_df.to_csv("sankey_activation.csv", index=False)

In [7]:
import pandas as pd

# 1. 데이터 불러오기
df = pd.read_csv("telco.csv")

# 2. 가입 시점 추정
df['EstimatedJoinMonth'] = pd.to_datetime("2023-01-01") - pd.to_timedelta(df['Tenure in Months'] * 30, unit='D')
df['JoinCohort'] = df['EstimatedJoinMonth'].dt.to_period('M').dt.to_timestamp()

# 3. 상/하반기 Cohort 그룹 만들기
def get_half_year_label(date):
    year = date.year
    if date.month <= 6:
        return f"{year} 상반기 가입자"
    else:
        return f"{year} 하반기 가입자"

df['CohortGroup'] = df['JoinCohort'].apply(get_half_year_label)

# 4. Sankey용 구조 생성
df_sankey = df.groupby(['CohortGroup', 'Churn Label']).agg(
    value=('Customer ID', 'count')
).reset_index()

# 5. 컬럼명 정리
df_sankey.columns = ['source', 'target', 'value']

# 6. 저장
df_sankey.to_csv("sankey_joincohort_halfyear.csv", index=False)
print("✅ 저장 완료: sankey_joincohort_halfyear.csv")


✅ 저장 완료: sankey_joincohort_halfyear.csv


In [8]:
from lifelines import KaplanMeierFitter
import pandas as pd

df['ChurnBinary'] = df['Churn Label'].map({'Yes': 1, 'No': 0})

T = df['Tenure in Months']
E = df['ChurnBinary']

kmf = KaplanMeierFitter()
kmf.fit(durations=T, event_observed=E, label='전체 고객')

# 생존 곡선 추출 (월 단위)
survival_df = kmf.survival_function_.reset_index()
survival_df.columns = ['TenureMonth', 'SurvivalProbability']
survival_df['Cohort'] = '전체 고객'

# 저장
survival_df.to_csv("retention_curve_monthly.csv", index=False)


In [9]:
# 위에서 만든 survival_df 사용
import numpy as np

# 분기 단위 구간 설정
survival_df['TenureQuarter'] = pd.cut(
    survival_df['TenureMonth'],
    bins=[0, 3, 6, 9, 12, 15, 18, 21, 24, 36, 48, 60, 72],
    labels=[
        '0~3개월', '4~6개월', '7~9개월', '10~12개월', '13~15개월', '16~18개월',
        '19~21개월', '22~24개월', '25~36개월', '37~48개월', '49~60개월', '61개월+'
    ],
    right=False
)

# 각 구간별 평균 잔존율 계산
summary_df = survival_df.groupby('TenureQuarter').agg(
    SurvivalProbability=('SurvivalProbability', 'mean')
).reset_index()

summary_df.to_csv("retention_curve_summary.csv", index=False)


C:\Users\somee\AppData\Local\Temp\ipykernel_27460\4273383160.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary_df = survival_df.groupby('TenureQuarter').agg(
